In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import sklearn
import scipy as sp
import statsmodels as sm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision as tv
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.utils as vutils
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss

In [2]:
# Data are sorted in ./SuperData
Train_DataX = pd.read_csv("./SuperData/X_train.csv", index_col=0, header=[0,1,2])
Valid_DataX = pd.read_csv("./SuperData/X_valid.csv", index_col=0, header=[0,1,2])
X_Test = pd.read_csv('./SuperData/X_test.csv', index_col=0 , header=[0,1,2])
Train_DataY = pd.read_csv("./task2/Y_train.csv", index_col=0)
Valid_DataY = pd.read_csv("./task2/Y_valid.csv", index_col=0)

# Fill nan with 0
Train_DataX = Train_DataX.fillna(0)
Valid_DataX = Valid_DataX.fillna(0)
X_Test = X_Test.fillna(0)

In [3]:
# print number of rows in Train_DataX
print("Train_DataX has", len(Train_DataX), "rows")

Train_DataX has 16760 rows


In [4]:
# Show size of Train_DataX after dropping columns
Train_DataX.shape

(16760, 7904)

In [5]:
Train_DataX.head()

LEVEL2               alanine aminotransferase                                \
Aggregation Function                     mask                                 
hours_in                                    0    1    2    3    4    5    6   
3_145834_211552                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
6_107064_228232                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_150750_220597                           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11_194540_229441                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
12_112213_232669                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   

LEVEL2                               ... white blood cell count urine         \
Aggregation Function                 ...          time_since_measured          
hours_in                7    8    9  ...                           15     16   
3_145834_211552       0.0  0.0  0.0  ...                          7.0    8.0   
6_107064_228232       0.0  0.0  0.0  ...                         31.0   32.0   
9_150750_220597       0.0  0.0  0.0  ...                         55.0   56.0   
11_194540_229441      0.0  0.0  0.0  ...                         79.0   80.0   
12_112213_232669      0.0  0.0  0.0  ...                        103.0  104.0   

LEVEL2                                                                        
Aggregation Function                                                          
hours_in                 17     18     19     20     21     22     23   diff  
3_145834_211552         9.0   10.0   11.0   12.0   13.0   14.0   15.0  100.0  
6_107064_228232        33.0   34.0   35.0   36.0   37.0   38.0   39.0   23.0  
9_150750_220597        57.0   58.0   59.0   60.0   61.0   62.0   63.0   23.0  
11_194540_229441       81.0   82.0   83.0   84.0   85.0   86.0   87.0   23.0  
12_112213_232669      105.0  106.0  107.0  108.0  109.0  110.0  111.0   23.0  

[5 rows x 7904 columns]

In [6]:
# Change each to numpy array
X_train = Train_DataX.to_numpy()
X_valid = Valid_DataX.to_numpy()
X_test = X_Test.to_numpy()
Y_train = Train_DataY.to_numpy()
Y_valid = Valid_DataY.to_numpy()

In [7]:
X_train.shape

(16760, 7904)

In [8]:
Y_train = Y_train.ravel()
Y_valid = Y_valid.ravel()
Y_train.shape

(16760,)

In [9]:
# First regression model
# fast 4 min
from sklearn.linear_model import LinearRegression
model = LinearRegression(n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 9197.378572


In [ ]:
# Second regression model
# fast 3min
from sklearn.linear_model import Ridge
model = Ridge(alpha=0.1, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 173.363853


In [ ]:
# Third regression model
# fast 2min
from sklearn.linear_model import Lasso
model = Lasso(alpha=0.1, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.824610


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.736e+04, tolerance: 6.543e+00
  model = cd_fast.enet_coordinate_descent(


In [6]:
# Fourth regression model
# fast 5min
from sklearn.linear_model import ElasticNet
model = ElasticNet(alpha=0.1)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.816818


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.677e+04, tolerance: 6.543e+00
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Fifth regression model, use non-linear model
# shit 81 min
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=300, max_depth=10, random_state=3612, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.747301


In [ ]:
# Sixth regression model, use non-linear model
# shit 62min
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(n_estimators=300, max_depth=5, random_state=3612, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.761030


In [ ]:
# Seventh regression model, use non-linear model
# Fast 3min
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(n_estimators=300, random_state=3612)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.962370


In [ ]:
# Eighth regression model, use non-linear model
# Fast 6min
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=300, max_depth=10, random_state=3612, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.753345


In [ ]:
# Ninth regression model, use non-linear model
# shit 66min
from sklearn.ensemble import BaggingRegressor
model = BaggingRegressor(n_estimators=300, random_state=3612, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred)) 
print("RMSE: %f" % (rmse))

RMSE: 1.750438


In [ ]:
# Tenth regression model, use non-linear model
# normal 10min
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators=300, max_depth=10, random_state=3612, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.841207


In [9]:
# Eleventh regression model, use non-linear model
# KNN1
# Fast 5.1s
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=1, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 2.668084


In [12]:
# Twelveth regression model, use non-linear model
# KNN10
# Fast 4.5s
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=10, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 2.056702


In [13]:
# Thirteenth regression model, use non-linear model
# KNN100
# Fast 3.9s
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=100, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 2.003364


In [14]:
# Fourteenth regression model, use non-linear model
# KNN1000
# Fast 5.7s
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=1000, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.994576


In [16]:
# Fifteenth regression model, use non-linear model
# KNN10000
# Fast 3.9s
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=10000, n_jobs=5)
model.fit(X_train, Y_train)
Y_valid_pred = model.predict(X_valid)
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.994970


In [ ]:
# Train X_train and Y_train by Ensemble model on the skewed data
from sklearn.ensemble import VotingClassifier
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
# HistGradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
# XGBoost
import xgboost as xgb

model = xgb.XGBRegressor(booster='gbtree', eval_metric='rmse', n_estimators=1000,
                            max_depth=3, min_child_weight=4,
                            learning_rate=0.04, subsample=0.6, colsample_bytree=1, random_state=3314, n_jobs=5)
model.fit(X_train, Y_train)

# Valid by X_valid and Y_valid
Y_valid_pred = model.predict(X_valid)
# Get rmse
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_valid, Y_valid_pred))
print("RMSE: %f" % (rmse))

RMSE: 1.739007


In [ ]:
# if auc < 1.73, output the result by X_test
if rmse < 1.73:
    Y_test_prob = model.predict(X_test)
    index = X_Test.index
    output = pd.DataFrame(Y_test_prob, columns=['Predicted'], index=index, dtype=float)
    print(output.head())
    output.to_csv('output.csv', index_label='Id')